In [ ]:
import numpy as np
import pandas as pd

df_train = pd.read_csv('/content/Data_train.csv')
df_test = pd.read_csv('/content/Data_test.csv')

df_train

,type,group,education,meal,preparation course,score-1,score-2,score-3
0,wild,group B,some high school,standard,completed,63,67,67
1,domestic,group C,bachelor's degree,standard,none,67,69,75
2,domestic,group C,some college,standard,completed,69,90,88
3,domestic,group B,some high school,standard,none,62,64,66
4,wild,group E,some college,reduced,none,93,90,83
...,...,...,...,...,...,...,...,...
695,wild,group C,some high school,standard,completed,67,73,68
696,domestic,group B,high school,reduced,none,8,24,23
697,wild,group E,some college,standard,completed,85,75,68
698,wild,group E,associate's degree,standard,none,53,45,40


In [ ]:
df_test.isna().sum().sum()
df_train.isna().sum().sum()

20

In [ ]:
num_columns = df_train.select_dtypes(include=[np.number]).columns
cat_columns = df_train.select_dtypes(include=[object]).columns


In [ ]:
for column in num_columns:
    mean_value = df_train[column].mean()
    df_train[column].fillna(mean_value, inplace=True)

for column in cat_columns:
    unique_category = 'Unique'
    df_train[column].fillna(unique_category, inplace=True)

In [ ]:
num_columns = df_test.select_dtypes(include=[np.number]).columns
cat_columns = df_test.select_dtypes(include=[object]).columns

for column in num_columns:
    mean_value = df_train[column].mean()
    df_test[column].fillna(mean_value, inplace=True)

for column in cat_columns:
    unique_category = 'Unique'
    df_test[column].fillna(unique_category, inplace=True)

In [ ]:
df_train['Pass'] = (df_train['score-1'] > 50) & (df_train['score-2'] > 50) & (df_train['score-3'] > 50)

In [ ]:
df_train['Pass'] = df_train['Pass'].astype(int)
num_not_passed = len(df_train) - df_train['Pass'].sum()

In [ ]:
num_not_passed

145

In [ ]:
grouped = df_train.groupby('type')['Pass'].mean()

diff = abs(grouped['wild'] - grouped['domestic'])
diff = round(diff, 2)  # округление до сотых

In [ ]:
diff

0.02

In [ ]:
df_train

,type,group,education,meal,preparation course,score-1,score-2,score-3,Pass
0,wild,group B,some high school,standard,completed,63,67,67,1
1,domestic,group C,bachelor's degree,standard,none,67,69,75,1
2,domestic,group C,some college,standard,completed,69,90,88,1
3,domestic,group B,some high school,standard,none,62,64,66,1
4,wild,group E,some college,reduced,none,93,90,83,1
...,...,...,...,...,...,...,...,...,...
695,wild,group C,some high school,standard,completed,67,73,68,1
696,domestic,group B,high school,reduced,none,8,24,23,0
697,wild,group E,some college,standard,completed,85,75,68,1
698,wild,group E,associate's degree,standard,none,53,45,40,0


In [ ]:
instructor_level = 'high school'
not_passed = df_train['Pass'] == 0

num_cats = len(df_train[not_passed & (df_train['education'] == instructor_level)])

In [ ]:
num_cats

35

In [ ]:
cat_type = 'wild'
passed = df_train['Pass'] == 1
no_course = df_train['preparation course'] == 'none'

num_cats = len(df_train[passed & (df_train['type'] == cat_type) & no_course])

In [ ]:
num_cats

152

In [ ]:
median_score = df_train['score-1'].median()

In [ ]:
median_score

66.0

In [ ]:
domestic_no_course = df_train[(df_train['type'] == 'domestic') & (df_train['preparation course'] == 'none')]

third_quartile = domestic_no_course['score-3'].quantile(0.75, interpolation='lower')
first_quartile = domestic_no_course['score-3'].quantile(0.25, interpolation='lower')

iqr = third_quartile - first_quartile

In [ ]:
iqr

20

In [ ]:
cat_cols = df_train.select_dtypes(include=[object]).columns

df_train_encoded = pd.get_dummies(df_train, columns=cat_cols, drop_first=True)
df_test_encoded = pd.get_dummies(df_test, columns=cat_cols, drop_first=True)

In [ ]:
num_encoded_cols = df_train_encoded.shape[1] - (df_train.shape[1] - len(cat_cols))

In [ ]:
num_encoded_cols

13

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

# Создаем матрицу объект-признак X и вектор ответов y
X = df_train_encoded.drop('Pass', axis=1)  # предполагаем, что целевой столбец называется 'Pass'
y = df_train_encoded['Pass']

# Создаем и настраиваем модель дерева решений
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5)

# Выполняем кросс-валидацию и выводим средний roc-auc
score = cross_val_score(clf, X, y, cv=3, scoring='roc_auc').mean()
rounded_score = round(score, 1)

print(f'Средний ROC-AUC: {rounded_score}')

Средний ROC-AUC: 1.0


In [ ]:
from sklearn.model_selection import GridSearchCV

# Задаем диапазон значений для max_depth
param_grid = {'max_depth': range(2, 21)}

# Создаем модель решающего дерева
clf = DecisionTreeClassifier(criterion='entropy')

grid_search_clf = GridSearchCV(clf, param_grid, cv=3, scoring='roc_auc')

# Обучаем модель на закодированных данных
grid_search_clf.fit(X, y)

# Выводим лучшее значение max_depth
print(f'Лучшее значение max_depth: {grid_search_clf.best_params_["max_depth"]}')

Лучшее значение max_depth: 4


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# Creating new feature 'cat_bio'
df_train['cat_bio'] = df_train[['type', 'group']].apply(lambda x: f'({x[0]}, {x[1]})', axis=1)

# Applying OneHotEncoder to the specified columns
encoder = OneHotEncoder(drop='first')
encoded_cols = encoder.fit_transform(df_train[['cat_bio', 'education', 'meal', 'preparation course']]).toarray()

# Convert the numpy array back into a DataFrame
df_encoded = pd.DataFrame(encoded_cols)

# Adding the target variable to df_encoded
df_encoded['Pass'] = df_train['Pass'].values

# Splitting the data into features and target variable
X = df_encoded.drop(['Pass'], axis=1)
y = df_encoded['Pass']

# Creating and fitting the model
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5)
clf.fit(X, y)

# Computing and printing ROC-AUC
score = cross_val_score(clf, X, y, cv=3, scoring='roc_auc').mean()
rounded_score = round(score, 2)
print(rounded_score)

0.68


In [ ]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Создаём новый признак 'cat_bio'
df_train['cat_bio'] = df_train[['type', 'group']].apply(lambda x: f'({x[0]}, {x[1]})', axis=1)
df_test['cat_bio'] = df_test[['type', 'group']].apply(lambda x: f'({x[0]}, {x[1]})', axis=1)

# Применяем OneHotEncoder к указанным столбцам
encoder = OneHotEncoder(drop='first')
encoded_cols_train = encoder.fit_transform(df_train[['cat_bio', 'education', 'meal', 'preparation course']]).toarray()
encoded_cols_test = encoder.transform(df_test[['cat_bio', 'education', 'meal', 'preparation course']]).toarray()

# Преобразовываем numpy array обратно в DataFrame и добавляем целевую переменную
df_encoded_train = pd.DataFrame(encoded_cols_train, columns = encoder.get_feature_names(['cat_bio', 'education', 'meal', 'preparation course']))
df_encoded_train['Pass'] = df_train['Pass']

df_encoded_test = pd.DataFrame(encoded_cols_test, columns = encoder.get_feature_names(['cat_bio', 'education', 'meal', 'preparation course']))

# Создаём матрицу объект-признак X и вектор ответов y
X_train = df_encoded_train.drop('Pass', axis=1)
y_train = df_encoded_train['Pass']

# Создаём и обучаем модель
xg_clf = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 100)
xg_clf.fit(X_train, y_train)

# Предсказываем вероятности классов для тренировочного набора и вычисляем ROC_AUC
train_preds_proba = xg_clf.predict_proba(X_train)[:,1]
roc_auc = roc_auc_score(y_train, train_preds_proba)
print(f'Train ROC_AUC: {roc_auc}')

# Предсказываем вероятности классов для тестового набора
test_preds_proba = xg_clf.predict_proba(df_encoded_test)[:,1]

# Сохраняем предсказанные вероятности в файл
np.savetxt("result.txt", test_preds_proba, fmt="%f")

AttributeError: 'OneHotEncoder' object has no attribute 'get_feature_names'